# __1. 탐색적 데이터 분석__

##### 파이썬 패키지 import

In [1]:
%matplotlib inline

from pathlib import Path

import pandas as pd
import numpy as np
from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles

import seaborn as sns
import matplotlib.pylab as plt

## __1.1 정형화된 데이터의 요소__

#### 수치형 : 숫자를 이용해 표현할 수 있는 데이터
#### 연속형 : 일정 범위 안에서 어떤 값이든 취할 수 있는 데이터
#### 이산 : 정수 값만 취할 수 있는 데이터
#### 범주형 : 가능하나 범주 안의 값만을 취하는 데이터 (=명목형)
#### 이진 : 두 개의 값만을 갖는 범주형 데이터의 특수한 경우
#### 순서형 : 값들 사이에 분명한 순위가 있는 범주형 데이터

## __1.2 테이블 데이터__

#### 테이블 데이터 : 스프레드시트와 같은 테이블 형테의 데이터 구조(이차원 행렬)
#### 피처 : 일반적으로 테이블의 각 열
#### 결과 : 데이터 과학 프로젝트의 목표: 어떠한 결과를 예측, 결과 예측을 위해 피처 사용
#### 레코드 : 일반적으로 테이블의 각 행

## __1.3 위치 추정__

### __1.3.1 평균__

#### 평균 : 모든 값을 총합의 개수로 나눈 값
## $\bar{x} = \frac{\sum^n_{i=1}{x}_i}{n}$

#### 절사평균 : 정해진 개수의 극단값을 제외한 나머지 값들의 평균
#### p : 가장 크고 작은 값들의 개수
## $\bar{x} = \frac{\sum^{n-p}_{i=p+1} {x}_{(i)}}{n-2p}$

#### 가중평균 : 데이터를 정렬한 후, 각 가중치 값을 위에서부터 더할 때, 총합의 중간이 위치하는 데이터 값
#### ${w_i} =$ 사용자가 지정한 가중치
## $\bar{x_i} = \frac{\sum^n_{i=1}w_i x_i}{\sum^n_{i=1} {w_i}}$

### __1.3.2 중간값과 로버스트 추정__

#### 중간값 : 데이터에서 가장 가운데 위치한 값
#### 가중 중간값 : 데이터를 정렬한 후, 각 가중치 값을 위에서부터 더할 때, 총합의 중간이 위치하는 데이터 값
#### 로버스트하다 : 극단값에 민감하지 않다
#### 특잇값(outliner) : 대부분의 값과 매우 다른 데이터 값(=극단값)

### __예제 : 인구에 따른 살인 비율의 위치 추정__

In [7]:
state = pd.read_csv('./data/state.csv')
print(state['Population'].mean()) # 평균
print(trim_mean(state['Population'],0.1)) # 절사평균
print(state['Population'].median()) # 중간값

6162876.3
4783697.125
4436369.5


In [10]:
# 가중평균

print(np.average(state['Murder.Rate'], weights=state['Population'])) # 가중평균
print(wquantiles.median(state['Murder.Rate'], weights=state['Population'])) # 가중 중간값

4.445833981123393
4.4


## __1.4 변이 추정__

#### 편차 : 관측값과 위치 추정값 사이의 차이 (=잔차)
#### 평균절대편차 : 편차의 절댓값의 평균
## 평균절대편차 = $\frac{\sum_{i=1}^{n} \left\vert{x_i-\bar{x}}\right\vert}{n}$

#### 분산 : 잔차의 제곱의 평균, 잔차르르 제곱한 값을 n-1으로 나눈 값
## $s^2 = \frac{\sum^n_{i=1}(x_i-\bar{x})^2}{n-1}$

#### 표준편차 : 분산의 제곱근

## $s = \sqrt{분산}$

#### __자유도__
#### 분산을 구할 떄 왜 n-1을 분모로 사용하는 가?

#### 전제 : 표본을 가지고 모집단을 추정하고자 함
#### 분산 수식에 n을 분모로 사용한다면, 모집단의 분산과 표준편차의 참값을 과소평가하게 됨 (=__편향 추정__)

#### n을 사용하는 것이 편향 추정이 되는 이유
#### 자유도 : 추정값을 계산할 때 제약 조건의 개수
#### __표준편차는 표본의 평균을 따른다는 제약 조건이 있기 때문에 n-1의 자유도를 가진다.__

#### 중간값의 중위절대편차(MAD)

### $MAD = 중간값(\left\vert x_1-m \right\vert), (\left\vert x_2-m \right\vert),\cdots, (\left\vert x_N-m \right\vert)$

### __1.4.2 백분위에 기초한 추정__

#### 순서통계량 : 정렬 데이터를 나타내는 통계량
#### 기본 측도 : 가장 큰 값과 가장 작은 값의 차이를 나타내는 범위

#### 백분위수 : 전체 데이터의 P%를 아래에 두는 값
#### 데이터에서 P번째 백분위수는 P퍼센트의 값이 그 값, 그보다 작은 값을 갖고(100-P)퍼센트의 값이 그 값 혹은 그보다 큰 값을 갖는 어떤 값
#### 사분위범위(IQR) : 25번째 백분위수와 75번째 백분위수의 차이

### __예제 : 주별 인구의 변이 추정__

In [11]:
# pandas 메서드 활용

state['Population'].std() # 표준편차

6848235.347401142

In [14]:
state['Population'].quantile(0.75) - state['Population'].quantile(0.25) # IQR

4847308.0

In [15]:
robust.scale.mad(state['Population']) # MAD

3849876.1459979336